In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
import numpy as np
np.random.seed(10)
(m, n) = (30, 10)
A = np.random.rand(m, n); A = np.asmatrix(A)
b = np.random.rand(m, 1); b = np.asmatrix(b)
c_nom = np.ones((n, 1)) + np.random.rand(n, 1); c_nom = np.asmatrix(c_nom)

In [10]:
F = np.vstack([np.eye(n), -np.eye(n), np.ones((1, n)) / n, -np.ones((1, n)) / n])
g = np.vstack([ 1.25 * c_nom, -0.75 * c_nom, 1.1*np.sum(c_nom) / n, -0.9 * np.sum(c_nom) / n])

In [26]:
x = cp.Variable((10, 1))
lam = cp.Variable((22, 1))

cost = cp.Minimize(g.T @ lam)
constraints = [
    F.T @ lam == x, 
    lam >= 0,
    A @ x >= b
]

problem = cp.Problem(cost, constraints)
problem.solve()
x_robust = x.value
print(f'x_robust: \n{x.value}')
print(f'Robust estimate(c_nom^T @ x):{c_nom.T @ x_robust}')

# Computing f(x) the dual problem
worst_case_cost = cp.Problem(cp.Minimize(g.T @ lam), [F.T @ lam == x_robust, lam >= 0]).solve()
print(f'Worst case cost with robust estimate:{worst_case_cost}')

x_robust: 
[[ 0.10226062]
 [ 0.10226062]
 [ 0.48813484]
 [ 0.38758877]
 [ 0.10226062]
 [ 0.33331083]
 [ 0.10226062]
 [ 0.10226062]
 [ 0.12340546]
 [-0.16020985]]
Robust estimate(c_nom^T @ x):[[2.52320886]]
Worst case cost:3.1659610489081342


### Non robust estimate

In [28]:
x = cp.Variable((10, 1))
non_robust_cost = cp.Problem(cp.Minimize(c_nom.T @ x), [A @ x >= b]).solve()
print(f"Non-robust estimate:{non_robust_cost}")

x_non_robust = x.value

# Computing f(x) the dual problem
worst_case_cost = cp.Problem(cp.Minimize(g.T @ lam), [F.T @ lam == x_non_robust, lam >= 0]).solve()
print(f'Worst case cost with non-robust estimate:{worst_case_cost}')

Non-robust estimate:2.1092714620826003
Worst case cost with non-robust estimate:7.22156220310519
